# Sparkify DataLakes AWS and Spark Project
This notebook is meant to testing purposes
**by jerryespn**
-April 2020-

### Library import

In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

### Configuration File for AWS

In [2]:
config = configparser.ConfigParser()
config.read('conf/dl.cfg')

['conf/dl.cfg']

### AWS access keys taken from configuration file

In [3]:
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

### SparkContext Initialize

In [4]:
def create_spark_session():
    '''
    Initialize the SparkSession for the application, includes AWS package to use it
    '''
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark
spark = create_spark_session()

### Process Song Function

In [ ]:
###### TESTING THE QUERIES #####

In [5]:
spark = create_spark_session()

In [6]:
#song_data = "s3://udacity-dend/song_data/*/*/*"
song_data = "data/song_data/*/*/*"

In [7]:
df = spark.read.json(song_data)

In [ ]:
df.take(10)

In [ ]:
df.createOrReplaceTempView("songs")

In [ ]:
song_table = (df.dropna(how = "any", subset = ["song_id", "artist_id"]) \
                    .where(df["song_id"].isNotNull()) \
                    .select(["song_id", "title", "artist_id", "year", "duration"]) \
                    .dropDuplicates()) \
    (song_table.write.mode("overwrite") \
                     .partitionBy("year", "artist_id") \
                     .parquet(output_data + "songs.parquet"))

In [ ]:
song_table = spark.sql ("SELECT DISTINCT song_id, title as song_title, artist_id, year, duration FROM songs")

In [ ]:
song_table.take(10)

In [ ]:
song_table.write.mode("overwrite").partitionBy("year", "artist_id").parquet(output_data)

### Process Log Function

In [ ]:
def process_log_data(spark, input_data, output_data):
    '''
    This function extracts and process the log data from S3 into DataFrames,
    :input_data -> S3 Bucket where data comes
    :output_data -> S3 Bucket where data is stored
    '''
    # get filepath to log data file
    log_data =

    # read log data file
    df = 
    
    # filter by actions for song plays
    df = 

    # extract columns for users table    
    artists_table = 
    
    # write users table to parquet files
    artists_table

    # create timestamp column from original timestamp column
    get_timestamp = udf()
    df = 
    
    # create datetime column from original timestamp column
    get_datetime = udf()
    df = 
    
    # extract columns to create time table
    time_table = 
    
    # write time table to parquet files partitioned by year and month
    time_table

    # read in song data to use for songplays table
    song_df = 

    # extract columns from joined song and log datasets to create songplays table 
    songplays_table = 

    # write songplays table to parquet files partitioned by year and month
    songplays_table

In [ ]:
# Spark Session start
spark = create_spark_session()

In [ ]:
# Data Aquisition
input_data = "s3a://udacity-dend/"
output_data = ""

In [ ]:
# Process Data
process_song_data(spark, input_data, output_data)    
process_log_data(spark, input_data, output_data)